# Make Python code installable

## Dependency hazard

- Dependencies quickly sum up
- Especially version requirements lead to conflicts

## Environments isolate dependencies

- Most common options (more exist):
  - conda
  - venv
- Conda environments are easier to access from anywhere
- Venv environments are easier to set up
- UV manages venv making rapid creation and teardown possible
- Pixi (not in the current scope) is like uv but more comprehensive

# Make code modular

A first step to sharing code is identifying reusable pieces of code.

> If you ctrl-c + ctrl-p, consider defining variables, functions, classes

## Exercise: custom plot function

Below follow a sample script containing repeating lines to produce three
similar plots. Let's wrap them in a function.

In [ ]:
# ChatGPT example code (explicitly linear)

import requests
import matplotlib.pyplot as plt
import datetime as dt
import sys

# Vienna coordinates and timezone
lat = 48.2085
lon = 16.3721
tz  = "Europe/Vienna"

# --------------------------
# Fetch weather
# --------------------------
url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": lat,
    "longitude": lon,
    "hourly": "temperature_2m,relativehumidity_2m,windspeed_10m",
    "timezone": tz,
}

try:
    r = requests.get(url, params=params, timeout=15)
    r.raise_for_status()
except Exception as e:
    print("Failed to fetch data:", e, file=sys.stderr)
    sys.exit(1)

try:
    data = r.json()
except Exception as e:
    print("Failed to parse JSON:", e, file=sys.stderr)
    sys.exit(1)

if "hourly" not in data:
    print("API returned no hourly data", file=sys.stderr)
    sys.exit(1)

hourly = data["hourly"]

# --------------------------
# Extract variables
# --------------------------
raw_times = hourly.get("time", [])
temps = hourly.get("temperature_2m", [])
hums  = hourly.get("relativehumidity_2m", [])
winds = hourly.get("windspeed_10m", [])

if not (raw_times and temps and hums and winds):
    print("Missing required variables in API response.", file=sys.stderr)
    sys.exit(1)

# --------------------------
# Convert times
# --------------------------
times = []
for t in raw_times:
    try:
        times.append(dt.datetime.fromisoformat(t))
    except:
        try:
            times.append(dt.datetime.fromisoformat(t.replace("Z", "+00:00")))
        except Exception as e:
            print("Bad time format:", t, e, file=sys.stderr)
            sys.exit(1)

In [ ]:
# --------------------------
# Plot everything
# --------------------------
fig, axes = plt.subplots(3, 1, sharex=True, figsize=(10, 8))
fig.suptitle("Vienna – Hourly Weather Forecast (Open-Meteo)")

axes[0].plot(times, temps)
axes[0].set_ylabel("Temp [°C]")
axes[0].grid(True, linestyle="--", linewidth=0.5)

axes[1].plot(times, hums)
axes[1].set_ylabel("RH [%]")
axes[1].grid(True, linestyle="--", linewidth=0.5)

axes[2].plot(times, winds)
axes[2].set_ylabel("Wind [km/h]")
axes[2].grid(True, linestyle="--", linewidth=0.5)
plt.show()

In [ ]:
# def custom_plot

# axes[0].plot(times, temps)
# axes[0].set_ylabel("Temp [°C]")
# axes[0].grid(True, linestyle="--", linewidth=0.5)

In [ ]:
# placeholder: optional outsource exercise. mind local variable `times`

In [ ]:
fig, axes = plt.subplots(3, 1, sharex=True, figsize=(10, 8))
fig.suptitle("Vienna – Hourly Weather Forecast (Open-Meteo)")

for ax, data, name, units in zip(axes, [temps, hums, winds], ["Temp", "RH", "Wind"], ["°C", "%", "km/h"]):
    custom_plot(ax, data, name, units)

plt.show()

## Learnings check-up and room for questions

## Subtle differences: App, Library, Package

### App

Focussed on exporting interfaces that allow accomplishing specific tasks.

* Narrow restrictions to ensure expected behavior
* Usually no API (application programming interface)
* Best used in own environment

### Library

Opposite of App: focussed on exporting an API; versatile

* Restrictions as broad as possible, as narrow as necessary
* Usually no CLI/GUI (command line-/graphical user interface)
* Made to be integrated in foreign environments

### Package

Hybrid of App and Library: exports API and CLI or GUI

**However, structure more similar to Library**

* Dependencies often free within major versions (the first version identifier is fixed)
* Made to be integrated in foreign environments

### Opinion: use Package/Library structures, especially for sharing code